In [15]:
import pandas as pd

df = pd.read_csv("../cleaned_data/user_with_cnt_coupon_weather_mgm_cleaned.csv")

pd.set_option("display.max_columns", None)
df.head()

,experiment_date,treatment,source,ops_type_merged,city,user_cnt,nonrepeat_cnt,trip_cnt,weekday_nonrepeat_cnt,weekend_nonrepeat_cnt,weekday_trip_cnt,weekend_trip_cnt,nonrepeat_cnt_per_user,trip_cnt_per_user,weekday_nonrepeat_cnt_per_user,weekend_nonrepeat_cnt_per_user,weekday_trip_cnt_per_user,weekend_trip_cnt_per_user,weekday_match_rate,weekend_match_rate,coupon_BD_total,coupon_CDP_total,coupon_folk_total,coupon_growth_other_total,coupon_MGM_total,coupon_MKT_total,coupon_register_total,coupon_daily_total,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day
0,2025-07-28,不發,控制組,14天在其他尖峰預估車資,南投縣,3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,NaN,NaN,0,3,0,0,0,0,0,0,1.000000,0.0,0.5,7.0
1,2025-07-28,不發,控制組,14天在其他尖峰預估車資,嘉義市,15,4.0,4.0,1.0,3.0,0.0,4.0,0.266667,0.266667,0.066667,0.20,0.00,0.266667,NaN,1.00,0,0,0,0,0,0,0,0,1.000000,0.0,1.0,7.0
2,2025-07-28,不發,控制組,14天在其他尖峰預估車資,嘉義縣,13,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,NaN,NaN,0,3,0,0,0,0,0,0,1.000000,0.0,0.5,7.0
3,2025-07-28,不發,控制組,14天在其他尖峰預估車資,基隆市,50,13.0,10.0,6.0,7.0,7.0,3.0,0.260000,0.200000,0.120000,0.14,0.14,0.060000,0.89,0.75,0,28,0,0,4,0,0,0,0.714286,0.0,0.0,7.0
4,2025-07-28,不發,控制組,14天在其他尖峰預估車資,宜蘭縣,12,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,NaN,NaN,0,3,0,0,0,0,0,0,0.857143,0.0,0.0,7.0


In [16]:
import pandas as pd
import numpy as np


df["experiment_date"] = pd.to_datetime(df["experiment_date"])

north = {'臺北市','新北市','基隆市','桃園市','宜蘭縣','花蓮縣','新竹縣','新竹市'}
central = {'苗栗縣','臺中市','南投縣','彰化縣','雲林縣','嘉義縣','嘉義市'}
south = {'臺南市','高雄市','屏東縣','臺東縣','澎湖縣','金門縣','連江縣'}

def map_city_group(city):
    if pd.isna(city) or city == "":
        city = "臺北市"
    if city in north:
        return "北區"
    if city in central:
        return "中區"
    if city in south:
        return "南區"
    return "北區"

df["city_group"] = df["city"].apply(map_city_group)


In [17]:
def wavg(x, value_col, weight_col):
    v = x[value_col]
    w = x[weight_col]
    mask = v.notna() & w.notna() & (w > 0)
    if mask.sum() == 0:
        return np.nan
    return (v[mask] * w[mask]).sum() / w[mask].sum()


In [18]:
GROUP_KEYS = ["experiment_date", "treatment", "source", "ops_type_merged", "city_group"]

SUM_COLS = [
    # 人數
    "user_cnt",
    # 趟次（非重複、trip）
    "weekday_nonrepeat_cnt", "weekend_nonrepeat_cnt", "nonrepeat_cnt",
    "weekday_trip_cnt", "weekend_trip_cnt", "trip_cnt",
    # 券張數
    "coupon_BD_total", "coupon_CDP_total", "coupon_folk_total", "coupon_growth_other_total",
    "coupon_MGM_total", "coupon_MKT_total", "coupon_register_total", "coupon_daily_total",
]

MEAN_COLS = [
    # 天氣（你資料已是 avg_ 開頭）
    "avg_rainy_day", "avg_rainy_weekday", "avg_rainy_weekend",
    # mgm_day（同一天通常一樣，mean/max都可）
    "mgm_day",
]

# 先聚合 sum / mean
agg_dict = {c: "sum" for c in SUM_COLS}
agg_dict.update({c: "mean" for c in MEAN_COLS})

out = df.groupby(GROUP_KEYS, as_index=False).agg(agg_dict)

# 重算人均欄位（用總量 / user_cnt）
out["nonrepeat_cnt_per_user"] = out["nonrepeat_cnt"] / out["user_cnt"]
out["trip_cnt_per_user"] = out["trip_cnt"] / out["user_cnt"]

out["weekday_nonrepeat_cnt_per_user"] = out["weekday_nonrepeat_cnt"] / out["user_cnt"]
out["weekend_nonrepeat_cnt_per_user"] = out["weekend_nonrepeat_cnt"] / out["user_cnt"]

out["weekday_trip_cnt_per_user"] = out["weekday_trip_cnt"] / out["user_cnt"]
out["weekend_trip_cnt_per_user"] = out["weekend_trip_cnt"] / out["user_cnt"]


In [19]:
if {"weekday_total", "weekend_total"}.issubset(df.columns):
    mr = df.groupby(GROUP_KEYS).apply(
        lambda x: pd.Series({
            "weekday_match_rate": wavg(x, "weekday_match_rate", "weekday_total"),
            "weekend_match_rate": wavg(x, "weekend_match_rate", "weekend_total"),
        })
    ).reset_index()
else:
    mr = df.groupby(GROUP_KEYS).apply(
        lambda x: pd.Series({
            "weekday_match_rate": wavg(x, "weekday_match_rate", "weekday_trip_cnt"),
            "weekend_match_rate": wavg(x, "weekend_match_rate", "weekend_trip_cnt"),
        })
    ).reset_index()
out = out.merge(mr, on=GROUP_KEYS, how="left")


C:\Users\minhsiang.chang\AppData\Local\Temp\ipykernel_18160\4035954074.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mr = df.groupby(GROUP_KEYS).apply(


In [20]:
# out 已經沒有 city（因為 group keys 沒放 city）
# 欄位順序整理（可依你要的報告/建模順序調）
COL_ORDER = GROUP_KEYS + [
    "user_cnt",
    "nonrepeat_cnt", "trip_cnt",
    "weekday_nonrepeat_cnt", "weekend_nonrepeat_cnt",
    "weekday_trip_cnt", "weekend_trip_cnt",
    "nonrepeat_cnt_per_user", "trip_cnt_per_user",
    "weekday_nonrepeat_cnt_per_user", "weekend_nonrepeat_cnt_per_user",
    "weekday_trip_cnt_per_user", "weekend_trip_cnt_per_user",
    "weekday_match_rate", "weekend_match_rate",
    "coupon_BD_total", "coupon_CDP_total", "coupon_folk_total", "coupon_growth_other_total",
    "coupon_MGM_total", "coupon_MKT_total", "coupon_register_total", "coupon_daily_total",
    "avg_rainy_day", "avg_rainy_weekday", "avg_rainy_weekend",
    "mgm_day",
]

# 有些欄位可能不存在（例如 match_rate 你可能還沒算進去），安全處理：
COL_ORDER = [c for c in COL_ORDER if c in out.columns]
out = out[COL_ORDER]

out.head()

,experiment_date,treatment,source,ops_type_merged,city_group,user_cnt,nonrepeat_cnt,trip_cnt,weekday_nonrepeat_cnt,weekend_nonrepeat_cnt,weekday_trip_cnt,weekend_trip_cnt,nonrepeat_cnt_per_user,trip_cnt_per_user,weekday_nonrepeat_cnt_per_user,weekend_nonrepeat_cnt_per_user,weekday_trip_cnt_per_user,weekend_trip_cnt_per_user,weekday_match_rate,weekend_match_rate,coupon_BD_total,coupon_CDP_total,coupon_folk_total,coupon_growth_other_total,coupon_MGM_total,coupon_MKT_total,coupon_register_total,coupon_daily_total,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day
0,2025-07-28,不發,控制組,14天在其他尖峰預估車資,中區,333,91.0,45.0,74.0,17.0,34.0,11.0,0.273273,0.135135,0.222222,0.051051,0.102102,0.033033,0.523529,0.749091,20,114,0,2,9,0,24,0,0.897959,0.000000,0.500000,7.0
1,2025-07-28,不發,控制組,14天在其他尖峰預估車資,北區,1220,228.0,161.0,165.0,63.0,129.0,32.0,0.186885,0.131967,0.135246,0.051639,0.105738,0.026230,0.793101,0.779062,107,583,0,12,299,0,30,1,0.732143,0.000000,0.000000,7.0
2,2025-07-28,不發,控制組,14天在其他尖峰預估車資,南區,374,73.0,51.0,52.0,21.0,36.0,15.0,0.195187,0.136364,0.139037,0.056150,0.096257,0.040107,0.751944,0.850000,30,129,1,2,10,1,21,0,0.809524,0.366667,0.583333,7.0
3,2025-07-28,不發,控制組,14天在晚尖峰預估車資,中區,106,34.0,23.0,23.0,11.0,15.0,8.0,0.320755,0.216981,0.216981,0.103774,0.141509,0.075472,0.744000,0.917500,10,36,0,2,1,0,0,0,0.897959,0.000000,0.500000,7.0
4,2025-07-28,不發,控制組,14天在晚尖峰預估車資,北區,288,56.0,42.0,48.0,8.0,37.0,5.0,0.194444,0.145833,0.166667,0.027778,0.128472,0.017361,0.845676,0.750000,37,90,0,2,56,0,6,0,0.732143,0.000000,0.000000,7.0


In [21]:
out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3072 entries, 0 to 3071
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   experiment_date                 3072 non-null   datetime64[ns]
 1   treatment                       3072 non-null   object        
 2   source                          3072 non-null   object        
 3   ops_type_merged                 3072 non-null   object        
 4   city_group                      3072 non-null   object        
 5   user_cnt                        3072 non-null   int64         
 6   nonrepeat_cnt                   3072 non-null   float64       
 7   trip_cnt                        3072 non-null   float64       
 8   weekday_nonrepeat_cnt           3072 non-null   float64       
 9   weekend_nonrepeat_cnt           3072 non-null   float64       
 10  weekday_trip_cnt                3072 non-null   float64       
 11  week

In [22]:
out["mgm_day"] = out["mgm_day"].fillna(0).astype(int)

# 2) （建議）把應該是整數的欄位轉回 int，避免 0.0 看起來怪
INT_COLS = [
    "user_cnt",
    "weekday_nonrepeat_cnt","weekend_nonrepeat_cnt","nonrepeat_cnt",
    "weekday_trip_cnt","weekend_trip_cnt","trip_cnt",
    "coupon_BD_total","coupon_CDP_total","coupon_folk_total","coupon_growth_other_total",
    "coupon_MGM_total","coupon_MKT_total","coupon_register_total","coupon_daily_total",
    "mgm_day",
]
INT_COLS = [c for c in INT_COLS if c in out.columns]
out[INT_COLS] = out[INT_COLS].astype(int)


In [23]:
out.head()

,experiment_date,treatment,source,ops_type_merged,city_group,user_cnt,nonrepeat_cnt,trip_cnt,weekday_nonrepeat_cnt,weekend_nonrepeat_cnt,weekday_trip_cnt,weekend_trip_cnt,nonrepeat_cnt_per_user,trip_cnt_per_user,weekday_nonrepeat_cnt_per_user,weekend_nonrepeat_cnt_per_user,weekday_trip_cnt_per_user,weekend_trip_cnt_per_user,weekday_match_rate,weekend_match_rate,coupon_BD_total,coupon_CDP_total,coupon_folk_total,coupon_growth_other_total,coupon_MGM_total,coupon_MKT_total,coupon_register_total,coupon_daily_total,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day
0,2025-07-28,不發,控制組,14天在其他尖峰預估車資,中區,333,91,45,74,17,34,11,0.273273,0.135135,0.222222,0.051051,0.102102,0.033033,0.523529,0.749091,20,114,0,2,9,0,24,0,0.897959,0.000000,0.500000,7
1,2025-07-28,不發,控制組,14天在其他尖峰預估車資,北區,1220,228,161,165,63,129,32,0.186885,0.131967,0.135246,0.051639,0.105738,0.026230,0.793101,0.779062,107,583,0,12,299,0,30,1,0.732143,0.000000,0.000000,7
2,2025-07-28,不發,控制組,14天在其他尖峰預估車資,南區,374,73,51,52,21,36,15,0.195187,0.136364,0.139037,0.056150,0.096257,0.040107,0.751944,0.850000,30,129,1,2,10,1,21,0,0.809524,0.366667,0.583333,7
3,2025-07-28,不發,控制組,14天在晚尖峰預估車資,中區,106,34,23,23,11,15,8,0.320755,0.216981,0.216981,0.103774,0.141509,0.075472,0.744000,0.917500,10,36,0,2,1,0,0,0,0.897959,0.000000,0.500000,7
4,2025-07-28,不發,控制組,14天在晚尖峰預估車資,北區,288,56,42,48,8,37,5,0.194444,0.145833,0.166667,0.027778,0.128472,0.017361,0.845676,0.750000,37,90,0,2,56,0,6,0,0.732143,0.000000,0.000000,7


In [24]:
out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3072 entries, 0 to 3071
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   experiment_date                 3072 non-null   datetime64[ns]
 1   treatment                       3072 non-null   object        
 2   source                          3072 non-null   object        
 3   ops_type_merged                 3072 non-null   object        
 4   city_group                      3072 non-null   object        
 5   user_cnt                        3072 non-null   int64         
 6   nonrepeat_cnt                   3072 non-null   int64         
 7   trip_cnt                        3072 non-null   int64         
 8   weekday_nonrepeat_cnt           3072 non-null   int64         
 9   weekend_nonrepeat_cnt           3072 non-null   int64         
 10  weekday_trip_cnt                3072 non-null   int64         
 11  week

In [25]:
out.describe()

,experiment_date,user_cnt,nonrepeat_cnt,trip_cnt,weekday_nonrepeat_cnt,weekend_nonrepeat_cnt,weekday_trip_cnt,weekend_trip_cnt,nonrepeat_cnt_per_user,trip_cnt_per_user,weekday_nonrepeat_cnt_per_user,weekend_nonrepeat_cnt_per_user,weekday_trip_cnt_per_user,weekend_trip_cnt_per_user,weekday_match_rate,weekend_match_rate,coupon_BD_total,coupon_CDP_total,coupon_folk_total,coupon_growth_other_total,coupon_MGM_total,coupon_MKT_total,coupon_register_total,coupon_daily_total,avg_rainy_day,avg_rainy_weekday,avg_rainy_weekend,mgm_day
count,3072,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,2790.000000,2721.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000,3072.000000
mean,2025-11-22 19:07:30,510.999674,69.468424,49.097005,47.664714,21.803711,30.503906,18.593099,0.136437,0.099833,0.094065,0.042372,0.061906,0.037928,0.833127,0.803915,84.898438,158.760742,396.679362,3.155599,9.760742,5.014323,8.642578,0.125977,0.182280,0.068039,0.014371,1.695312
min,2025-07-28 00:00:00,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.220000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2025-11-03 00:00:00,126.000000,11.000000,8.000000,8.000000,3.000000,5.000000,3.000000,0.071073,0.050894,0.047437,0.018443,0.028294,0.017391,0.764538,0.719412,19.000000,18.000000,64.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2025-11-24 00:00:00,259.000000,30.000000,22.000000,21.000000,9.000000,13.000000,8.000000,0.114125,0.083423,0.078845,0.034524,0.049489,0.031447,0.840000,0.807500,45.000000,54.000000,212.500000,0.000000,0.000000,0.000000,2.000000,0.000000,0.040816,0.000000,0.000000,0.000000
75%,2025-12-22 00:00:00,608.500000,73.000000,53.000000,49.000000,23.000000,32.000000,21.000000,0.194904,0.145196,0.135802,0.060092,0.091317,0.053441,0.910500,0.900000,104.000000,165.000000,504.000000,3.000000,2.000000,5.000000,10.000000,0.000000,0.357143,0.025000,0.000000,4.000000
max,2026-01-12 00:00:00,9388.000000,1248.000000,931.000000,815.000000,506.000000,606.000000,343.000000,0.651741,0.531250,0.546875,0.232558,0.406250,0.279070,1.000000,1.000000,1508.000000,3736.000000,4414.000000,162.000000,2945.000000,109.000000,274.000000,8.000000,0.964286,0.964286,0.875000,7.000000
std,NaN,723.010386,118.604486,81.692020,81.661050,38.534251,52.474465,31.028287,0.092370,0.068480,0.065607,0.033926,0.047117,0.029710,0.117735,0.141545,121.318474,313.678952,537.973044,8.810269,64.921773,10.843960,17.540181,0.467173,0.237089,0.167116,0.071156,2.171034


In [26]:
from pathlib import Path

out_path = Path("..") / "final_data" / "data_260116.csv"
out.to_csv(out_path, index=False, encoding="utf-8-sig")

print("Saved:", out_path.resolve())

Saved: D:\minhsiang.chang\Desktop\2026winter_project\final_data\data_260116.csv
